### Задание 1.
Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы

In [5]:
!pip install pyconll

In [79]:
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm

2022-09-05 15:33:59.104931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-09-05 15:33:59.104979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-05 15:34:06.445900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-09-05 15:34:06.445987: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-05 15:34:06.453797: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-GL267VQB
2022-09-05 15:34:06.454106: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-GL267VQB


============================== Info about spaCy ==============================

spaCy version    3.4.1                         
Location         C:\Users\Uliana\anaconda3\lib\site-packages\spacy
Platform         Windows-10-10.0.19041-SP0     
Python version   3.8.8                         
Pipelines                                      

     ---------------------------------------- 15.3/15.3 MB 7.7 MB/s eta 0:00:00

2022-09-05 15:34:13.845614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-09-05 15:34:13.845682: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-05 15:34:21.257591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-09-05 15:34:21.257673: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-05 15:34:21.265545: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-GL267VQB
2022-09-05 15:34:21.265840: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-GL267VQB



[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [84]:
!pip install navec

In [89]:
!pip install slovnet

     -------------------------------------- 49.4/49.4 kB 833.4 kB/s eta 0:00:00


In [93]:
!pip install ipymarkup

In [113]:
!pip install sklearn_crfsuite

     -------------------------------------- 152.5/152.5 kB 1.3 MB/s eta 0:00:00


In [132]:
import pyconll
import pandas as pd
import os
import csv
import nltk
import spacy
from spacy import displacy
import ru_core_news_sm
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from nltk.tokenize import word_tokenize
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from corus import load_ne5

In [38]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [11]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [12]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [13]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [15]:
fdata_train[:3]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [18]:
unigram_tagger = UnigramTagger(fdata_train)
acc_u = unigram_tagger.evaluate(fdata_test)
display(acc_u)

<ipython-input-18-c0c1d20b533d>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_u = unigram_tagger.evaluate(fdata_test)


0.823732013802982

In [23]:
bigram_tagger = BigramTagger(fdata_train)
acc_b = bigram_tagger.evaluate(fdata_test)
display(acc_b)

<ipython-input-23-8fa1fd4878f1>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_b = bigram_tagger.evaluate(fdata_test)


0.6093886320724006

In [24]:
trigram_tagger = TrigramTagger(fdata_train)
acc_t = trigram_tagger.evaluate(fdata_test)
display(acc_t)

<ipython-input-24-921b35dd23f0>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_t = trigram_tagger.evaluate(fdata_test)


0.1778631421316492

In [25]:
combination_1 = [BigramTagger, TrigramTagger]

combination_2 = [TrigramTagger, UnigramTagger]

combination_3 = [UnigramTagger, BigramTagger]

combination_4 = [BigramTagger, TrigramTagger, UnigramTagger]

In [28]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

acc_bo = []

for combination in [combination_1, combination_2, combination_3, combination_4]:
    backoff = DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         combination,  
                         backoff = backoff) 

    acc_bo.append(tag.evaluate(fdata_test))

acc_bo

<ipython-input-28-e3565fadf4c9>:14: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_bo.append(tag.evaluate(fdata_test))


[0.1750309264926102,
 0.21248779217396965,
 0.8275343446838986,
 0.3623152548994075]

In [31]:
result = pd.DataFrame({'Tagger': ['UnigramTagger', 'BigramTagger', 'TrigramTagger', 'BigramTagger & TrigramTagger', 'TrigramTagger & UnigramTagger', 'UnigramTagger & BigramTagger', 'All taggers'], 'Accuracy' : [acc_u, acc_b, acc_t] + acc_bo})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
5,UnigramTagger & BigramTagger,0.827534
0,UnigramTagger,0.823732
1,BigramTagger,0.609389
6,All taggers,0.362315
4,TrigramTagger & UnigramTagger,0.212488
2,TrigramTagger,0.177863
3,BigramTagger & TrigramTagger,0.175031


In [32]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [35]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [42]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
hvectorizer = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
hvectorizer_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]

In [51]:
f1_scores = []
acc_scores = []

for vectorizer in vectorizers + vectorizers_word + hvectorizer + hvectorizer_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    acc_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.90      0.91      0.91     11247
         ADP       0.99      0.99      0.99     10255
         ADV       0.92      0.86      0.89      5986
         AUX       0.81      0.97      0.89      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.80      0.83      0.82      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.90      0.96      0.93     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.85      0.90      0.87      1436
        PART       0.95      0.79      0.86      3762
        PRON       0.87      0.84      0.86      5346
       PROPN       0.80      0.52      0.63      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.64      0.78        53
        VERB       0.93     

C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for 

...

C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         ADJ       0.63      0.33      0.44     11247
         ADP       0.91      0.48      0.63     10255
         ADV       0.82      0.72      0.77      5986
         AUX       0.79      0.95      0.86      1058
       CCONJ       0.92      0.19      0.31      4276
         DET       0.69      0.70      0.70      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.70      0.62      0.65     27241
      NO_TAG       0.00      0.00      0.00       197
         NUM       0.66      0.48      0.55      1436
        PART       0.91      0.76      0.83      3762
        PRON       0.79      0.79      0.79      5346
       PROPN       0.54      0.13      0.21      4315
       PUNCT       0.45      1.00      0.62     21941
       SCONJ       0.76      0.88      0.82      2176
         SYM       0.00      0.00      0.00        53
        VERB       0.67      0.36      0.47     12617
           X       0.00    

C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


HashingVectorizer(n_features=10000, ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.72      0.36      0.48     11247
         ADP       0.96      0.48      0.64     10255
         ADV       0.88      0.75      0.81      5986
         AUX       0.81      0.88      0.84      1058
       CCONJ       0.88      0.20      0.33      4276
         DET       0.72      0.75      0.74      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.77      0.64      0.70     27241
      NO_TAG       0.00      0.00      0.00       197
         NUM       0.74      0.58      0.65      1436
        PART       0.96      0.75      0.84      3762
        PRON       0.83      0.78      0.81      5346
       PROPN       0.67      0.15      0.24      4315
       PUNCT       0.42      1.00      0.60     21941
       SCONJ       0.78      0.88      0.83      2176
         SYM       0.00      0.00      0.00        53
        VERB       0.75  

C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Uliana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + hvectorizer + hvectorizers_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927836
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921158
9,"HashingVectorizer(analyzer='char', n_features=...",0.903654
8,"HashingVectorizer(analyzer='char', n_features=...",0.901213
7,"HashingVectorizer(analyzer='char', n_features=...",0.897075
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882215
4,"TfidfVectorizer(ngram_range=(1, 3))",0.662784
3,"CountVectorizer(ngram_range=(1, 3))",0.658415
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.620760


In [55]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + hvectorizer + hvectorizer_word,
                            'Accuracy': acc_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923583
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905009
7,"HashingVectorizer(analyzer='char', n_features=...",0.900913
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885157
4,"TfidfVectorizer(ngram_range=(1, 3))",0.653252
3,"CountVectorizer(ngram_range=(1, 3))",0.649052
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.630539


Судя по результатам комбинации таггеров лидирует по Accuracy смесь из UnigramTagger & BigramTagger с результатом - 0.827534
Во втором и третьем тестах лидирует CountVectorizer с f-score 0.927836 и accuracy 0.929513

### Задание 2. 
Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [62]:
fileDir = r"collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
text_list = []
for file in documents_txt:
    doc = open('collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text
0,Россия рассчитывает на конструктивное воздейст...
1,Комиссар СЕ критикует ограничительную политику...
2,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,4 октября назначены очередные выборы Верховног...
4,Следственное управление при прокуратуре требуе...
...,...
995,"Депутат от ""ЕР"": К отставке А.Сердюкова причас..."
996,\nСи Цзиньпин избран генсеком Коммунистической...
997,"""Ведомости"" узнали о смене лидера московских е..."
998,СМИ узнали о кутежах туркменского чиновника на...


In [65]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Uliana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Uliana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [66]:
document = data_text.text[0]

nltk.pos_tag(nltk.word_tokenize(document))


[('Россия', 'JJ'),
 ('рассчитывает', 'NNP'),
 ('на', 'NNP'),
 ('конструктивное', 'NNP'),
 ('воздействие', 'NNP'),
 ('США', 'NNP'),
 ('на', 'NNP'),
 ('Грузию', 'VBD'),
 ('04/08/2008', 'CD'),
 ('12:08', 'CD'),
 ('МОСКВА', 'NN'),
 (',', ','),
 ('4', 'CD'),
 ('авг', 'SYM'),
 ('-', ':'),
 ('РИА', 'NN'),
 ('Новости', 'NN'),
 ('.', '.'),
 ('Россия', 'JJ'),
 ('рассчитывает', 'NN'),
 (',', ','),
 ('что', 'NNP'),
 ('США', 'NNP'),
 ('воздействуют', 'NNP'),
 ('на', 'NNP'),
 ('Тбилиси', 'NNP'),
 ('в', 'NNP'),
 ('связи', 'NNP'),
 ('с', 'NNP'),
 ('обострением', 'NNP'),
 ('ситуации', 'NNP'),
 ('в', 'NNP'),
 ('зоне', 'NNP'),
 ('грузино-осетинского', 'JJ'),
 ('конфликта', 'NNP'),
 ('.', '.'),
 ('Об', 'VB'),
 ('этом', 'JJ'),
 ('статс-секретарь', 'JJ'),
 ('-', ':'),
 ('заместитель', 'NN'),
 ('министра', 'JJ'),
 ('иностранных', 'NNP'),
 ('дел', 'NNP'),
 ('России', 'NNP'),
 ('Григорий', 'NNP'),
 ('Карасин', 'NNP'),
 ('заявил', 'NNP'),
 ('в', 'NNP'),
 ('телефонном', 'NNP'),
 ('разговоре', 'NNP'),
 ('с', 'NNP

In [67]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Uliana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Uliana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [68]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('МИД России', 'ORGANIZATION'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('России Григорий Карасин', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Тбилиси', 'PERSON')}

In [69]:
pd.read_csv('collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


In [81]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [82]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Россия PROPN nsubj
рассчитывает VERB ROOT
на ADP case
конструктивное ADJ amod
воздействие NOUN obl
США PROPN nmod
на ADP case
Грузию PROPN nmod


 SPACE dep
04/08/2008 X appos
12:08 NUM appos


 SPACE dep
МОСКВА PROPN obj
, PUNCT punct
4 NUM conj
авг NOUN conj
- NOUN conj
РИА NOUN conj
Новости PROPN appos
. PUNCT punct
Россия PROPN nsubj
рассчитывает VERB ROOT
, PUNCT punct
что SCONJ mark
США PROPN nsubj
воздействуют VERB ccomp
на ADP case
Тбилиси PROPN obl
в ADP case
связи NOUN fixed
с ADP fixed
обострением NOUN obl
ситуации NOUN nmod
в ADP case
зоне NOUN nmod
грузино ADJ amod
- ADJ amod
осетинского ADJ amod
конфликта NOUN nmod
. PUNCT punct
Об ADP case
этом PRON obl
статс NOUN nsubj
- NOUN nsubj
секретарь NOUN nsubj
- NOUN nsubj
заместитель NOUN nsubj
министра NOUN nmod
иностранных ADJ amod
дел NOUN nmod
России PROPN nmod
Григорий PROPN appos
Карасин PROPN flat:name
заявил VERB ROOT
в ADP case
телефонном ADJ amod
разговоре NOUN obl
с ADP case
заместителем NOUN nmod
госсекретаря NOUN 

In [83]:
pd.read_csv('collection3/001.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [95]:
text = data_text.text[0]
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Россия рассчитывает на конструктивное воздействие США на Грузию
LOC───                                            LOC    LOC───
04/08/2008 12:08
МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют
LOC───          ORG────────  LOC───                   LOC             
 на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского 
    LOC────                                                           
конфликта. Об этом статс-секретарь - заместитель министра иностранных 
дел России Григорий Карасин заявил в телефонном разговоре с 
    LOC─── PER─────────────                                 
заместителем госсекретаря США Дэниэлом Фридом.
                          LOC PER──────────── 
"С российской стороны выражена глубокая озабоченность в связи с новым 
витком напряженности вокруг Южной Осетии, противозаконными действиями 
                            LOC─────────                              
грузинской стороны по наращиванию своих вооруженных сил в регионе, 
бес

In [96]:
fileDir = r"collection3"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['001.ann', '002.ann', '003.ann', '004.ann', '005.ann', '006.ann', '007.ann', '008.ann', '009.ann', '010.ann', '011.ann', '012.ann', '013.ann', '014.ann', '015 (!).ann', '016.ann', '017.ann', '018.ann', '019.ann', '020.ann', '021.ann', '022.ann', '023.ann', '025.ann', '026.ann', '027.ann', '028.ann', '029.ann', '030.ann', '031.ann', '032.ann', '033.ann', '034.ann', '035.ann', '036.ann', '037.ann', '038.ann', '039.ann', '03_12_12a.ann', '03_12_12b.ann', '03_12_12c.ann', '03_12_12d.ann', '03_12_12g.ann', '03_12_12h.ann', '040.ann', '041.ann', '042.ann', '043.ann', '044.ann', '045.ann', '046.ann', '047.ann', '048.ann', '049.ann', '04_02_13a_abdulatipov.ann', '04_03_13a_sorokin.ann', '04_12_12b.ann', '04_12_12d.ann', '04_12_12f.ann', '04_12_12g.ann', '04_12_12h_corr.ann', '050.ann', '051.ann', '052.ann', '053.ann', '054.ann', '055.ann', '056.ann', '057.ann', '058.ann', '059.ann', '060.ann', '061.ann', '062.ann', '063.ann', '064.ann', '065.ann', '066.ann', '067.ann', '068.ann', '069.ann', '

In [97]:
ann = pd.read_csv('collection3/001.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [101]:
docs = []
for i in range(len(data_text)):
    words = []
    labels = []
    text = data_text['text'][i]
    
    df = pd.read_csv('collection3/' + documents_ann[i], delimiter='\t', header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [102]:
print(docs[0][0]), print(docs[0][1])

['Россия', 'рассчитывает', 'на', 'конструктивное', 'воздействие', 'США', 'на', 'Грузию', '04/08/2008', '12', ':', '08', 'МОСКВА', ',', '4', 'авг', '-', 'РИА', 'Новости', '.', 'Россия', 'рассчитывает', ',', 'что', 'США', 'воздействуют', 'на', 'Тбилиси', 'в', 'связи', 'с', 'обострением', 'ситуации', 'в', 'зоне', 'грузино-осетинского', 'конфликта', '.', 'Об', 'этом', 'статс-секретарь', '-', 'заместитель', 'министра', 'иностранных', 'дел', 'России', 'Григорий', 'Карасин', 'заявил', 'в', 'телефонном', 'разговоре', 'с', 'заместителем', 'госсекретаря', 'США', 'Дэниэлом', 'Фридом', '.', '"', 'С', 'российской', 'стороны', 'выражена', 'глубокая', 'озабоченность', 'в', 'связи', 'с', 'новым', 'витком', 'напряженности', 'вокруг', 'Южной', 'Осетии', ',', 'противозаконными', 'действиями', 'грузинской', 'стороны', 'по', 'наращиванию', 'своих', 'вооруженных', 'сил', 'в', 'регионе', ',', 'бесконтрольным', 'строительством', 'фортификационных', 'сооружений', '"', ',', '-', 'говорится', 'в', 'сообщении', '

(None, None)

In [103]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Россия	LOC
рассчитывает	OUT
на	OUT
конструктивное	OUT
воздействие	OUT
США	LOC
на	OUT
Грузию	LOC
04/08/2008	OUT
12	OUT
:	OUT
08	OUT
МОСКВА	LOC
,	OUT
4	OUT
авг	OUT
-	OUT
РИА	ORG
Новости	ORG
.	OUT
Россия	LOC
рассчитывает	OUT
,	OUT
что	OUT
США	LOC
воздействуют	OUT
на	OUT
Тбилиси	LOC
в	OUT
связи	OUT
с	OUT
обострением	OUT
ситуации	OUT
в	OUT
зоне	OUT
грузино-осетинского	OUT
конфликта	OUT
.	OUT
Об	OUT
этом	OUT
статс-секретарь	OUT
-	OUT
заместитель	OUT
министра	OUT
иностранных	OUT
дел	OUT
России	LOC
Григорий	PER
Карасин	PER
заявил	OUT
в	OUT
телефонном	OUT
разговоре	OUT
с	OUT
заместителем	OUT
госсекретаря	OUT
США	LOC
Дэниэлом	PER
Фридом	PER
.	OUT
"	OUT
С	OUT
российской	OUT
стороны	OUT
выражена	OUT
глубокая	OUT
озабоченность	OUT
в	OUT
связи	OUT
с	OUT
новым	OUT
витком	OUT
напряженности	OUT
вокруг	OUT
Южной	LOC
Осетии	LOC
,	OUT
противозаконными	OUT
действиями	OUT
грузинской	OUT
стороны	OUT
по	OUT
наращиванию	OUT
своих	OUT
вооруженных	OUT
сил	OUT
в	OUT
регионе	OUT
,	OUT
бесконтрольным	OUT
строительс

In [104]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Россия,LOC
1,0,рассчитывает,OUT
2,0,на,OUT
3,0,конструктивное,OUT
4,0,воздействие,OUT
5,0,США,LOC
6,0,на,OUT
7,0,Грузию,LOC
8,0,04/08/2008,OUT
9,0,12,OUT


In [105]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)

sentences = getter.sentences

In [106]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [107]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [108]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X[2][7]

{'bias': 1.0,
 'word.lower()': 'в',
 'word[-3:]': 'в',
 'word[-2:]': 'в',
 'word.isdigit()': False,
 '-1:word.lower()': 'изъяты',
 '+1:word.lower()': 'арендуемом'}

In [109]:
len(X)

1000

In [110]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [115]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', # Градиентный спуск с использованием метода L-BFGS
    c1=0.1, # Коэффициент для регуляризации L1
    c2=0.1, # Коэффициент для регуляризации L2
    max_iterations=1000, # Максимальное количество итераций
    all_possible_transitions=True, # Генерация объектов (не встречающихся в обучающих данных)
    verbose=True # Включение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|████████████████████████████████████████████| 700/700 [00:03<00:00, 177.66it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 98963
Seconds required: 1.305

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.36  loss=121323.39 active=98091 feature_norm=1.00
Iter 2   time=0.16  loss=108888.46 active=94183 feature_norm=1.59
Iter 3   time=0.16  loss=106675.34 active=97456 feature_norm=1.47
Iter 4   time=0.16  loss=105543.37 active=97165 feature_norm=1.40
Iter 5   time=0.17  loss=103824.05 active=97821 feature_norm=1.46
Iter 6   time=0.25  loss=88396.99 active=97561 feature_norm=4.93
Iter 7   time=0.22  loss=86010.82 active=98482 feature_norm=4.83
Iter 8   time=0.26  loss=82187.81 active=98948 feature_norm=5.16
Iter 9   time=0.46  loss=78184.82 active=98894 feature_norm=6.23
Iter 10 

Iter 118 time=0.18  loss=14303.21 active=36370 feature_norm=194.28
Iter 119 time=0.16  loss=14302.89 active=36364 feature_norm=194.28
Iter 120 time=0.19  loss=14302.03 active=36349 feature_norm=194.30
Iter 121 time=0.18  loss=14301.47 active=36355 feature_norm=194.29
Iter 122 time=0.15  loss=14300.94 active=36345 feature_norm=194.31
Iter 123 time=0.23  loss=14300.57 active=36337 feature_norm=194.31
Iter 124 time=0.21  loss=14299.73 active=36344 feature_norm=194.33
Iter 125 time=0.19  loss=14299.27 active=36342 feature_norm=194.33
Iter 126 time=0.11  loss=14298.60 active=36338 feature_norm=194.34
Iter 127 time=0.28  loss=14298.14 active=36335 feature_norm=194.34
Iter 128 time=0.17  loss=14297.58 active=36318 feature_norm=194.36
Iter 129 time=0.16  loss=14297.16 active=36302 feature_norm=194.36
Iter 130 time=0.24  loss=14296.47 active=36293 feature_norm=194.38
Iter 131 time=0.21  loss=14296.00 active=36288 feature_norm=194.37
Iter 132 time=0.16  loss=14295.45 active=36269 feature_norm=19

Iter 243 time=0.25  loss=14270.24 active=35782 feature_norm=194.26
Iter 244 time=0.27  loss=14270.17 active=35781 feature_norm=194.26
Iter 245 time=0.23  loss=14270.12 active=35781 feature_norm=194.25
Iter 246 time=0.20  loss=14270.05 active=35775 feature_norm=194.25
Iter 247 time=0.12  loss=14270.01 active=35774 feature_norm=194.25
Iter 248 time=0.12  loss=14269.93 active=35771 feature_norm=194.25
Iter 249 time=0.19  loss=14269.89 active=35765 feature_norm=194.24
Iter 250 time=0.11  loss=14269.83 active=35761 feature_norm=194.24
Iter 251 time=0.13  loss=14269.78 active=35758 feature_norm=194.24
Iter 252 time=0.11  loss=14269.71 active=35756 feature_norm=194.24
Iter 253 time=0.19  loss=14269.66 active=35755 feature_norm=194.24
Iter 254 time=0.20  loss=14269.61 active=35747 feature_norm=194.23
Iter 255 time=0.24  loss=14269.56 active=35751 feature_norm=194.23
Iter 256 time=0.22  loss=14269.50 active=35746 feature_norm=194.23
Iter 257 time=0.23  loss=14269.46 active=35744 feature_norm=19

Iter 370 time=0.16  loss=14266.35 active=35602 feature_norm=194.12
Iter 371 time=0.17  loss=14266.32 active=35609 feature_norm=194.12
Iter 372 time=0.19  loss=14266.30 active=35607 feature_norm=194.12
Iter 373 time=0.20  loss=14266.28 active=35608 feature_norm=194.12
Iter 374 time=0.30  loss=14266.26 active=35609 feature_norm=194.12
Iter 375 time=0.18  loss=14266.24 active=35610 feature_norm=194.12
Iter 376 time=0.19  loss=14266.23 active=35611 feature_norm=194.12
Iter 377 time=0.27  loss=14266.19 active=35612 feature_norm=194.12
Iter 378 time=0.35  loss=14266.17 active=35614 feature_norm=194.12
Iter 379 time=0.17  loss=14266.16 active=35613 feature_norm=194.12
Iter 380 time=0.17  loss=14266.15 active=35612 feature_norm=194.12
Iter 381 time=0.16  loss=14266.11 active=35614 feature_norm=194.11
Iter 382 time=0.20  loss=14266.08 active=35616 feature_norm=194.11
Iter 383 time=0.16  loss=14266.05 active=35620 feature_norm=194.11
Iter 384 time=0.18  loss=14266.05 active=35622 feature_norm=19

In [116]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [120]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9464308202121121

In [121]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [122]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X[1][7]

{'bias': 1.0,
 'word.lower()': 'беженцев',
 'word[-3:]': 'цев',
 'word[-2:]': 'ев',
 'word.isdigit()': False}

In [123]:
len(X)

1000

In [124]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [125]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=True
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|████████████████████████████████████████████| 700/700 [00:02<00:00, 301.27it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 36195
Seconds required: 0.635

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.23  loss=121531.94 active=36022 feature_norm=1.00
Iter 2   time=0.13  loss=109078.51 active=35074 feature_norm=1.60
Iter 3   time=0.12  loss=106811.77 active=35898 feature_norm=1.47
Iter 4   time=0.15  loss=105847.31 active=35860 feature_norm=1.41
Iter 5   time=0.14  loss=104480.31 active=35923 feature_norm=1.45
Iter 6   time=0.11  loss=91312.47 active=35826 feature_norm=4.28
Iter 7   time=0.12  loss=89565.94 active=36097 feature_norm=4.59
Iter 8   time=0.11  loss=86624.69 active=36183 feature_norm=4.96
Iter 9   time=0.11  loss=85174.53 active=36154 feature_norm=6.23
Iter 10 

Iter 120 time=0.12  loss=20411.18 active=20752 feature_norm=204.82
Iter 121 time=0.11  loss=20408.90 active=20745 feature_norm=204.82
Iter 122 time=0.11  loss=20408.38 active=20736 feature_norm=204.80
Iter 123 time=0.12  loss=20406.23 active=20736 feature_norm=204.79
Iter 124 time=0.11  loss=20405.67 active=20731 feature_norm=204.75
Iter 125 time=0.11  loss=20403.41 active=20723 feature_norm=204.74
Iter 126 time=0.12  loss=20402.83 active=20724 feature_norm=204.71
Iter 127 time=0.12  loss=20400.88 active=20720 feature_norm=204.70
Iter 128 time=0.12  loss=20400.20 active=20733 feature_norm=204.66
Iter 129 time=0.13  loss=20398.42 active=20722 feature_norm=204.64
Iter 130 time=0.11  loss=20397.77 active=20712 feature_norm=204.61
Iter 131 time=0.12  loss=20395.97 active=20695 feature_norm=204.59
Iter 132 time=0.12  loss=20395.34 active=20696 feature_norm=204.55
Iter 133 time=0.11  loss=20393.78 active=20688 feature_norm=204.54
Iter 134 time=0.11  loss=20393.29 active=20684 feature_norm=20

Iter 244 time=0.12  loss=20354.43 active=20661 feature_norm=203.75
Iter 245 time=0.12  loss=20354.28 active=20656 feature_norm=203.75
Iter 246 time=0.12  loss=20354.19 active=20664 feature_norm=203.75
Iter 247 time=0.11  loss=20354.06 active=20667 feature_norm=203.75
Iter 248 time=0.11  loss=20353.99 active=20680 feature_norm=203.75
Iter 249 time=0.11  loss=20353.84 active=20683 feature_norm=203.75
Iter 250 time=0.12  loss=20353.78 active=20698 feature_norm=203.74
Iter 251 time=0.12  loss=20353.63 active=20697 feature_norm=203.74
Iter 252 time=0.12  loss=20353.56 active=20704 feature_norm=203.74
Iter 253 time=0.14  loss=20353.42 active=20704 feature_norm=203.74
Iter 254 time=0.12  loss=20353.37 active=20705 feature_norm=203.73
Iter 255 time=0.11  loss=20353.24 active=20703 feature_norm=203.73
Iter 256 time=0.11  loss=20353.18 active=20698 feature_norm=203.73
Iter 257 time=0.11  loss=20353.05 active=20698 feature_norm=203.73
Iter 258 time=0.11  loss=20352.98 active=20698 feature_norm=20

Iter 370 time=0.11  loss=20345.10 active=20567 feature_norm=203.62
Iter 371 time=0.12  loss=20344.99 active=20566 feature_norm=203.62
Iter 372 time=0.11  loss=20344.96 active=20567 feature_norm=203.62
Iter 373 time=0.12  loss=20344.82 active=20565 feature_norm=203.62
Iter 374 time=0.12  loss=20344.82 active=20566 feature_norm=203.62
Iter 375 time=0.13  loss=20344.70 active=20567 feature_norm=203.62
Iter 376 time=0.11  loss=20344.69 active=20565 feature_norm=203.62
Iter 377 time=0.12  loss=20344.56 active=20565 feature_norm=203.62
Iter 378 time=0.26  loss=20344.52 active=20566 feature_norm=203.62
Iter 379 time=0.22  loss=20344.47 active=20567 feature_norm=203.62
Iter 380 time=0.23  loss=20344.45 active=20567 feature_norm=203.62
Iter 381 time=0.22  loss=20344.39 active=20565 feature_norm=203.61
Iter 382 time=0.22  loss=20344.37 active=20566 feature_norm=203.61
Iter 383 time=0.26  loss=20344.30 active=20567 feature_norm=203.61
Iter 384 time=0.22  loss=20344.27 active=20565 feature_norm=20

Iter 494 time=0.22  loss=20339.86 active=20529 feature_norm=203.54
Iter 495 time=0.23  loss=20339.81 active=20527 feature_norm=203.54
Iter 496 time=0.23  loss=20339.80 active=20526 feature_norm=203.53
Iter 497 time=0.22  loss=20339.76 active=20525 feature_norm=203.53
Iter 498 time=0.25  loss=20339.75 active=20522 feature_norm=203.53
Iter 499 time=0.22  loss=20339.71 active=20519 feature_norm=203.53
Iter 500 time=0.22  loss=20339.69 active=20517 feature_norm=203.53
Iter 501 time=0.22  loss=20339.66 active=20517 feature_norm=203.53
Iter 502 time=0.23  loss=20339.64 active=20518 feature_norm=203.53
Iter 503 time=0.26  loss=20339.61 active=20520 feature_norm=203.53
Iter 504 time=0.22  loss=20339.59 active=20520 feature_norm=203.53
Iter 505 time=0.22  loss=20339.56 active=20518 feature_norm=203.53
Iter 506 time=0.23  loss=20339.54 active=20520 feature_norm=203.53
Iter 507 time=0.23  loss=20339.50 active=20519 feature_norm=203.53
Iter 508 time=0.24  loss=20339.49 active=20519 feature_norm=20

In [126]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [127]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9414778674716932